<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np

In [2]:
taxonomies_prod = pd.read_csv('/Users/alicia/Downloads/taxonomies_production.csv')
taxonomies_staging = pd.read_csv('/Users/alicia/Downloads/taxonomies_staging.csv')

In [3]:
taxonomies_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21548 entries, 0 to 21547
Data columns (total 17 columns):
id                     21548 non-null int64
class                  21539 non-null object
order                  21535 non-null object
suborder               0 non-null float64
superfamily            0 non-null float64
family                 21470 non-null object
subfamily              0 non-null float64
genus                  21066 non-null object
species                17409 non-null object
subspecies             3 non-null object
taxon_level            21548 non-null object
authority              17389 non-null object
common_name_english    20417 non-null object
iucn_category_id       17388 non-null float64
scientific_name        21046 non-null object
unique_identifier      21548 non-null object
taxonomy_type          21548 non-null object
dtypes: float64(4), int64(1), object(12)
memory usage: 2.8+ MB


In [4]:
taxonomies_staging.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21990 entries, 0 to 21989
Data columns (total 17 columns):
id                     21990 non-null int64
class                  21978 non-null object
order                  21973 non-null object
suborder               0 non-null float64
superfamily            0 non-null float64
family                 21908 non-null object
subfamily              0 non-null float64
genus                  21100 non-null object
species                17443 non-null object
subspecies             7 non-null object
taxon_level            21990 non-null object
authority              17410 non-null object
common_name_english    16769 non-null object
iucn_category_id       17420 non-null float64
scientific_name        21491 non-null object
taxonomy_type          535 non-null object
unique_identifier      21990 non-null object
dtypes: float64(4), int64(1), object(12)
memory usage: 2.9+ MB


In [23]:
for column in list(taxonomies_prod.columns):
    if column not in ['id', 'unique_identifier', 'common_name_english', 'scientific_name', 'authority']:
        print(column)
        result = set(taxonomies_prod[column].unique()) - set(taxonomies_staging[column].unique())
        print(list(result))

class
[]
family
[]
genus
[]
iucn_category_id
[nan, 2000001.0, 2000000.0, 2000003.0, 2000004.0, 2000005.0, 2000006.0, 2000007.0, 2000008.0, 2000010.0]
order
['Squamata']
species
['scrofa scrofa', 'gallus domesticus', 'aegagrus hircus', 'lupus familiaris']
subfamily
[nan]
suborder
[nan]
subspecies
[]
superfamily
[nan]
taxon_level
[]
taxonomy_type
[]


In [18]:
for column in list(taxonomies_staging.columns):
    print(column)
    print(taxonomies_staging[column].value_counts(dropna=False))

id
2047     1
8865     1
2708     1
661      1
6806     1
        ..
13628    1
15677    1
9534     1
11583    1
2049     1
Name: id, Length: 21990, dtype: int64
class
Aves        14622
Mammalia     7353
NaN            12
Unknown         1
Reptilia        1
Insecta         1
Name: class, dtype: int64
order
Passeriformes        8627
Rodentia             2854
Chiroptera           1494
Caprimulgiformes      807
Piciformes            601
                     ... 
Notoryctemorphia        6
Tubulidentata           5
Microbiotheria          5
Opisthocomiformes       5
Unknown                 1
Name: order, Length: 66, dtype: int64
suborder
NaN    21990
Name: suborder, dtype: int64
superfamily
NaN    21990
Name: superfamily, dtype: int64
family
Muridae             899
Cricetidae          835
Tyrannidae          558
Thraupidae          523
Psittacidae         500
                   ... 
Balaenicipitidae      4
Eurypygidae           4
Lipotidae             4
Antilocapridae        4
Unknown      

In [20]:
for column in list(taxonomies_prod.columns):
    print(column)
    print(taxonomies_prod[column].value_counts(dropna=False))

id
2000895    1
2013890    1
2001588    1
2005686    1
2003639    1
          ..
2002299    1
2012540    1
2014589    1
2008446    1
2000897    1
Name: id, Length: 21548, dtype: int64
class
Aves        14378
Mammalia     7157
NaN             9
Reptilia        2
Unknown         1
Insecta         1
Name: class, dtype: int64
order
Passeriformes        8490
Rodentia             2820
Chiroptera           1473
Caprimulgiformes      799
Piciformes            592
                     ... 
Tubulidentata           4
Microbiotheria          4
Opisthocomiformes       4
Unknown                 1
Squamata                1
Name: order, Length: 67, dtype: int64
suborder
NaN    21548
Name: suborder, dtype: int64
superfamily
NaN    21548
Name: superfamily, dtype: int64
family
Muridae           898
Cricetidae        834
Tyrannidae        557
Thraupidae        522
Psittacidae       499
                 ... 
Hypocoliidae        3
Diatomyidae         3
Scopidae            3
Antilocapridae      3
Unknown    

In [ ]:
taxonomies_prod.head()

In [ ]:
taxonomies_staging.head()

In [28]:
def diff_pd(df1, df2, cols):
    """Identify differences between two pandas DataFrames"""
    diffCols = set(df1.columns.values) ^ set(df2.columns.values)

    assert (set(df1.columns.values) ^ set(df2.columns.values)) is not None, \
        f"DataFrame column names are different: {diffCols}"
    
    df1.sort_index(axis=1, inplace=True)
    df2.sort_index(axis=1, inplace=True)
    
    print("Staging data shape"+ str(df1.shape))
    print("Prod data shape"+ str(df2.shape))
    if any(df1.dtypes != df2.dtypes):
        "Data Types are different, trying to convert"
        df2 = df2.astype(df1.dtypes)
    if df1.equals(df2):
        return None
    else:
        columns = set(df1.columns.values) - set(cols)
        print(columns)
        outputR = df1.merge(df2,how="right", on=['species', 'genus','family', 'order', 'taxon_level'], suffixes=('_staging', '_pro'), indicator=True)
        
        print(outputR._merge.value_counts(dropna=False))
        
        print('_________________________________')
        print(outputR.shape)
        print(outputR.info())
        outputRM = outputR[['suborder_pro', 'genus', 'family', 'superfamily_pro', 'subspecies_pro', 'scientific_name_pro', 'species', 'subfamily_pro', 'class_pro', 'order', 'taxonomy_type_pro', 'authority_pro', 'iucn_category_id_pro', 'common_name_english_pro', 'taxon_level', 'unique_identifier_pro','id_staging']]
        outputRM.drop_duplicates(keep=False)
        print(outputRM.info())
        
        return outputRM
    

In [29]:
test_output = diff_pd(taxonomies_staging, taxonomies_prod, ['unique_identifier','id'])

Staging data shape(21990, 17)
Prod data shape(21548, 17)
{'species', 'scientific_name', 'taxon_level', 'iucn_category_id', 'suborder', 'family', 'subspecies', 'superfamily', 'common_name_english', 'genus', 'order', 'taxonomy_type', 'subfamily', 'class', 'authority'}
both          22226
right_only        9
left_only         0
Name: _merge, dtype: int64
_________________________________
(22235, 30)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 22235 entries, 0 to 22234
Data columns (total 30 columns):
authority_staging              17408 non-null object
class_staging                  22130 non-null object
common_name_english_staging    17006 non-null object
family                         22051 non-null object
genus                          21243 non-null object
id_staging                     22226 non-null float64
iucn_category_id_staging       17416 non-null float64
order                          22118 non-null object
scientific_name_staging        21493 non-null object
species     

In [31]:
for column in list(test_output.columns):
    print(column)
    print(test_output[column].value_counts(dropna=True))

suborder_pro
Series([], Name: suborder_pro, dtype: int64)
genus
Crocidura      182
Homo           172
Myotis         113
Zosterops      101
Turdus          89
              ... 
Glironia         2
Nesoromys        2
Metopothrix      2
Falculea         2
Unknown          1
Name: genus, Length: 3658, dtype: int64
family
Muridae                899
Cricetidae             835
Tyrannidae             558
Thraupidae             523
Psittacidae            500
                      ... 
Hypsiprymnodontidae      4
Ornithorhynchidae        4
Microbiotheriidae        4
Antilocapridae           4
Unknown                  1
Name: family, Length: 405, dtype: int64
superfamily_pro
Series([], Name: superfamily_pro, dtype: int64)
subspecies_pro
hircus        1
domesticus    1
scrofa        1
Name: subspecies_pro, dtype: int64
scientific_name_pro
Homo sapiens (Linnaeus, 1758)                  19
Bubalus arnee (Kerr, 1792)                      3
Cervus canadensis (Erxleben, 1777)              3
Equus afric

In [32]:
r = test_output['unique_identifier_pro'].value_counts(dropna=False)

f = list(r.where(r>1).dropna().index)

s = test_output[test_output['unique_identifier_pro'].isin(f)]
s[s.taxon_level != 'none'].info()
s.drop_duplicates(subset=['unique_identifier_pro'], keep='first', inplace=True)
s

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 433 to 22164
Data columns (total 17 columns):
suborder_pro               0 non-null float64
genus                      196 non-null object
family                     1004 non-null object
superfamily_pro            0 non-null float64
subspecies_pro             0 non-null object
scientific_name_pro        34 non-null object
species                    196 non-null object
subfamily_pro              0 non-null float64
class_pro                  1028 non-null object
order                      1008 non-null object
taxonomy_type_pro          1028 non-null object
authority_pro              34 non-null object
iucn_category_id_pro       34 non-null float64
common_name_english_pro    1028 non-null object
taxon_level                1028 non-null object
unique_identifier_pro      1028 non-null object
id_staging                 1028 non-null float64
dtypes: float64(5), object(12)
memory usage: 144.6+ KB


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,suborder_pro,genus,family,superfamily_pro,subspecies_pro,scientific_name_pro,species,subfamily_pro,class_pro,order,taxonomy_type_pro,authority_pro,iucn_category_id_pro,common_name_english_pro,taxon_level,unique_identifier_pro,id_staging
433,NaN,Bos,Bovidae,NaN,NaN,"Bos mutus (Przewalski, 1883)",mutus,NaN,Mammalia,Cetartiodactyla,biological,"Przewalski, 1883",2000005.0,Domestic Yak,species,a914277e-da42-43f1-82e4-619a03542d9d,430.0
463,NaN,Bubalus,Bovidae,NaN,NaN,"Bubalus arnee (Kerr, 1792)",arnee,NaN,Mammalia,Cetartiodactyla,biological,"Kerr, 1792",2000004.0,Domestic Water Buffalo,species,c964ec4e-4ced-4e7c-9207-fbb81444d32a,457.0
555,NaN,Camelus,Camelidae,NaN,NaN,"Camelus ferus (Przewalski, 1878)",ferus,NaN,Mammalia,Cetartiodactyla,biological,"Przewalski, 1878",2000003.0,Bactrian Camel,species,94a98547-e316-4994-92b7-3d6810f7f3d3,546.0
684,NaN,Cervus,Cervidae,NaN,NaN,"Cervus canadensis (Erxleben, 1777)",canadensis,NaN,Mammalia,Cetartiodactyla,biological,"Erxleben, 1777",2000007.0,Domestic Elk,species,3e4fac86-1e90-403c-b798-c20a93df373c,671.0
1553,NaN,Equus,Equidae,NaN,NaN,"Equus africanus (Heuglin & Fitzinger, 1866)",africanus,NaN,Mammalia,Perissodactyla,biological,"Heuglin & Fitzinger, 1866",2000003.0,African Wild Ass,species,bf4af16d-d079-4d7c-af7a-be9c9c7cceab,1533.0
2076,NaN,Homo,Hominidae,NaN,NaN,"Homo sapiens (Linnaeus, 1758)",sapiens,NaN,Mammalia,Primates,biological,"Linnaeus, 1758",2000007.0,Human,species,990ae9dd-7a59-4344-afcb-1b7b21368000,2046.0
2095,NaN,Homo,Hominidae,NaN,NaN,NaN,sapiens,NaN,Mammalia,Primates,biological,NaN,NaN,Human-Camera Trapper,species,066c4187-4a03-4531-9f62-ac585ad0a138,2046.0
2114,NaN,Homo,Hominidae,NaN,NaN,NaN,sapiens,NaN,Mammalia,Primates,biological,NaN,NaN,Human-Hunter,species,f5ec47a5-e7a4-4158-a0b4-a6481ae3f06f,2046.0
2133,NaN,Homo,Hominidae,NaN,NaN,NaN,sapiens,NaN,Mammalia,Primates,biological,NaN,NaN,Human-Researcher,species,e6f88ed1-c305-4e94-b53d-8eec1b1d980a,2046.0
2152,NaN,Homo,Hominidae,NaN,NaN,NaN,sapiens,NaN,Mammalia,Primates,biological,NaN,NaN,Human-Tourist,species,795880f2-2677-41bf-be13-72c58056f08d,2046.0


In [33]:
test_output.drop_duplicates(subset=['unique_identifier_pro'], keep='first', inplace=True)

In [35]:
test_output.rename(columns={"suborder_pro": "suborder", "superfamily_pro": "superfamily", "subfamily_pro": "subfamily", "class_pro": "class", "authority_pro": "authority", "common_name_english_pro": "common_name_english", "iucn_category_id_pro": "iucn_category_id", "unique_identifier_pro": "unique_identifier","id_staging": "id","taxonomy_type_pro": "taxonomy_type","scientific_name_pro": "scientific_name","subspecies_pro": "subspecies"}, inplace=True)
test_output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21548 entries, 0 to 22234
Data columns (total 17 columns):
suborder               0 non-null float64
genus                  21066 non-null object
family                 21470 non-null object
superfamily            0 non-null float64
subspecies             3 non-null object
scientific_name        21046 non-null object
species                17409 non-null object
subfamily              0 non-null float64
class                  21539 non-null object
order                  21535 non-null object
taxonomy_type          21548 non-null object
authority              17389 non-null object
iucn_category_id       17388 non-null float64
common_name_english    20417 non-null object
taxon_level            21548 non-null object
unique_identifier      21548 non-null object
id                     21539 non-null float64
dtypes: float64(5), object(12)
memory usage: 3.0+ MB


In [36]:
test_output.to_csv('/Users/alicia/Downloads/taxonomies_result.csv',index=False)